In [ ]:
# Testing ReLU Activation

Quick experiment to test ReLU activation on speech commands.

ReLU is probably the most common activation function - it just zeros out negative values and passes positive values through unchanged.

## Data format
```
data/
├── yes/
├── no/
└── up/
```

Run the cells below to train and test a CNN with ReLU activations.


In [ ]:
# Install dependencies if needed
%pip install -r requirements.txt


In [ ]:
# Import libraries
import os
import json
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import librosa as lr
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

import tensorflow as tf
from tensorflow import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print("Activation: ReLU")


In [ ]:
# Configuration
DATASET_DIR = "/path/to/your/dataset"  # CHANGE THIS TO YOUR DATASET PATH
OUTPUT_BASE = "results"
SAMPLE_RATE = 16000
EPOCHS = 10
RANDOM_STATE = 44

print(f"Dataset directory: {DATASET_DIR}")
print(f"Output base directory: {OUTPUT_BASE}")
print(f"Sample rate: {SAMPLE_RATE}")
print(f"Training epochs: {EPOCHS}")
print(f"Activation: ReLU")

# Helper functions and data loading (same as master notebook)
def count_samples(path):
    sizes = [len(os.listdir(os.path.join(path, d))) for d in os.listdir(path)]
    return pd.DataFrame(sizes, index=os.listdir(path), columns=['num_samples'])

def load_dataset(path):
    data, labels, samples = [], [], []
    for label in os.listdir(path):
        dir_ = os.path.join(path, label)
        for fname in os.listdir(dir_):
            y, sr = lr.load(os.path.join(dir_, fname), sr=SAMPLE_RATE)
            data.append(y)
            samples.append(sr)
            labels.append(label)
    return data, labels, samples

def encode_labels(labels):
    code = {lab: i for i, lab in enumerate(sorted(set(labels)))}
    y = [code[lab] for lab in labels]
    return np.array(y), code

def build_model(layers, input_shape, num_classes):
    m = keras.Sequential()
    for i, L in enumerate(layers):
        t = L['type']
        if t == 'conv':
            kwargs = dict(filters=L['filters'], kernel_size=L['kernel_size'],
                          activation=L['activation'])
            if i == 0:
                m.add(keras.layers.Conv1D(input_shape=input_shape, **kwargs))
            else:
                m.add(keras.layers.Conv1D(**kwargs))
        elif t == 'pool':
            m.add(keras.layers.MaxPooling1D(pool_size=L['pool_size']))
        elif t == 'dropout':
            m.add(keras.layers.Dropout(rate=L['rate']))
        elif t == 'flatten':
            m.add(keras.layers.Flatten())
        elif t == 'dense':
            m.add(keras.layers.Dense(L['units'], activation=L['activation']))
    m.add(keras.layers.Dense(num_classes, activation='softmax'))
    m.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return m

def train_and_evaluate_model(model, layers, X_train, y_train, X_test, y_test):
    print("Configuration:")
    for L in layers:
        c = L.copy()
        if 'activation' in c and callable(c['activation']):
            c['activation'] = c['activation'].__name__
        print("  ", c)
    
    start = time.time()
    history = model.fit(X_train, y_train, epochs=EPOCHS,
                        validation_data=(X_test, y_test), verbose=2)
    elapsed = time.time() - start

    y_pred = np.argmax(model.predict(X_test), axis=-1)
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1_score': f1_score(y_test, y_pred, average='weighted'),
        'training_time': elapsed
    }
    return history, metrics

print("Functions defined.")


In [ ]:
# Load and prepare dataset
print("Loading dataset...")
data, labs, _ = load_dataset(DATASET_DIR)
y, label_map = encode_labels(labs)
X = np.array(data).reshape(-1, SAMPLE_RATE, 1)

print(f"Dataset shape: {X.shape}")
print(f"Number of classes: {len(label_map)}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE, shuffle=True
)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Run ReLU experiments
print("\n=== Running ReLU Experiments ===")

layers = [
    {'type': 'conv', 'filters': 16, 'kernel_size': 13, 'activation': tf.nn.relu},
    {'type': 'pool', 'pool_size': 3},
    {'type': 'dropout', 'rate': 0.3},
    {'type': 'conv', 'filters': 32, 'kernel_size': 11, 'activation': tf.nn.relu},
    {'type': 'pool', 'pool_size': 3},
    {'type': 'dropout', 'rate': 0.3},
    {'type': 'flatten'},
    {'type': 'dense', 'units': 128, 'activation': tf.nn.relu},
    {'type': 'dropout', 'rate': 0.3},
    {'type': 'dense', 'units': 64, 'activation': tf.nn.relu},
    {'type': 'dropout', 'rate': 0.3}
]

model = build_model(layers, input_shape=(SAMPLE_RATE, 1), num_classes=len(np.unique(y_train)))
history, metrics = train_and_evaluate_model(model, layers, X_train, y_train, X_test, y_test)

print(f"\nReLU Results:")
print(f"  Accuracy: {metrics['accuracy']:.3f}")
print(f"  F1 Score: {metrics['f1_score']:.3f}")
print(f"  Training Time: {metrics['training_time']:.1f}s")
